In [ ]:
import os
from regFilter.filter import *
from regFilter.spectra import *

In [ ]:
quant = 'feat_finding/features_010.csv'
samp = Filter(quant)
samp.formatFeatures()
samp.filterFeatures()
# Numeber of fearures: 82299
# Numeber of fearures after filtering: 1593

In [ ]:
samp.feat_tab.head()

In [ ]:
fnames = samp.feat_tab.columns
fnames = fnames[fnames.str.contains('Peak area')].str.replace(' Peak area', '').tolist()

In [ ]:
path = 'MS1/'
# Select small example of five features
stab = samp.feat_tab[samp.sel==1].head()

In [ ]:
if not os.path.exists('figs'):
    os.mkdir('figs')

for i in stab.index:
  overlayXIC(fnames, path, mz=stab.loc[i, 'row m/z'], rt=stab.loc[i, 'row retention time']*60, 
             fsz=10, type='TI', ppm=15, save=True, out=f"figs/{stab.loc[i,'row ID']}.png")

In [ ]:
from regFilter.create_report import *

In [ ]:
stab2 = stab[['row ID', 'row m/z', 'row retention time']].copy()
create_report(stab2)
# shold create regfilter_report.html file